# Quantitative Value Strategy

# Setup

In [1]:
# Numerical computing.
import numpy as np

# Tabular data manipulation.
import pandas as pd

# Requests.
import requests

# Statistics.
from scipy import stats

import xlsxwriter

import math

In [2]:
# API key.
from APIKey import APIKey

# Import List of Stocks

In [3]:
# sp_500_stocks.csv -- list of companies in S&P 500 (ticker symbol).
stocks = pd.read_csv('../sp_500_stocks.csv')

stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 1 columns):
Ticker    505 non-null object
dtypes: object(1)
memory usage: 4.1+ KB


In [4]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


# API

## Structure of an API Call

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={APIKey}'
data = requests.get(api_url).json()

data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'NKDAERSAQECMT(A BNOL SEL )GSG/LTA',
 'calculationPrice': 'close',
 'open': 0,
 'openTime': None,
 'openSource': 'flacifio',
 'close': 0,
 'closeTime': None,
 'closeSource': 'clfaifoi',
 'high': 0,
 'highTime': None,
 'highSource': None,
 'low': 0,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 145.86,
 'latestSource': 'Close',
 'latestTime': 'July 2, 2021',
 'latestUpdate': 1626506688660,
 'latestVolume': None,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 140.25,
 'previousVolume': 54381176,
 'change': 2.74,
 'changePercent': 0.0201,
 'volume': None,
 'iexMarketPercent': None,
 'iexVolume': None,
 'avgTotalVolume': 74215836,
 'iexBidPrice': Non

## Parsing Retrieved Data

In [6]:
pe_ratio = data['peRatio']
pe_ratio

32.88

# Simple Value Strategy

## Get the Required Data

### Execute Batch API Calls & Build a DataFrame

In [7]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]  

In [8]:
symbol_groups = list(chunks(stocks['Ticker'], 100))

symbol_groups

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

In [9]:
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [10]:
# Column names of required data.
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [11]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={APIKey}'
    
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 4 columns):
Ticker                     505 non-null object
Price                      505 non-null float64
Price-to-Earnings Ratio    492 non-null object
Number of Shares to Buy    505 non-null object
dtypes: float64(1), object(3)
memory usage: 15.9+ KB


In [12]:
final_dataframe.head()

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,151.84,49.8,N/A
1,AAL,21.97,-1.47,N/A
2,AAP,215.72,23.39,N/A
3,AAPL,143.51,32.61,N/A
4,ABBV,117.23,41.41,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock".

In [13]:
# Sort (ascending) by price to earnings ratio. 
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)

final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]

final_dataframe = final_dataframe[:50]

final_dataframe.reset_index(drop=True, inplace = True)

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,688.90,5.11,N/A
1,CINF,121.75,6.43,N/A
2,ZION,55.43,6.52,N/A
3,AFL,54.50,7.11,N/A
4,UNM,28.85,7.56,N/A
5,HRB,24.12,7.76,N/A
6,COO,414.52,8.76,N/A
7,CE,159.77,8.84,N/A
8,PGR,100.42,9,N/A
9,KIM,21.52,9.07,N/A


## Number of Shares to Buy

In [14]:
# Portfolio size = 1 million US$.
portfolio_size = 1000000

In [15]:
# Top 50 stocks are equally weighted.
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,688.90,5.11,29
1,CINF,121.75,6.43,164
2,ZION,55.43,6.52,360
3,AFL,54.50,7.11,366
4,UNM,28.85,7.56,693
5,HRB,24.12,7.76,829
6,COO,414.52,8.76,48
7,CE,159.77,8.84,125
8,PGR,100.42,9,199
9,KIM,21.52,9.07,929


## Export Recommended Trades

In [16]:
# Initializing XlsxWriter Object.
writer = pd.ExcelWriter('../recommended_trades/value_strategy_1.xlsx', engine='xlsxwriter')

final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [17]:
# Column formats.
# Background color.
background_color = '#0a0a23'

# Font color.
font_color = '#ffffff'

# String.
string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Price.
dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Integer.
integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Float.
float_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Percent.
percent_format = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
# Apply column formats.
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Price-to-Earnings Ratio', float_format],
                    'D': ['Number of Shares to Buy', integer_format],
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [19]:
# Save the excel file.
writer.save()

# Better Value Strategy

Every valuation metric has certain flaws. For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings. Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.
Investors typically use a composite basket of valuation metrics to build robust quantitative value strategies. 

In this section, we will filter for stocks with the lowest percentiles on the following metrics:
* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

## Get the Required Data

### Sample API Call

In [20]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={APIKey}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

### Execute Batch API Calls & Build a DataFrame

In [21]:
# Column names of required data.
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={APIKey}'
    
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [22]:
rv_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 14 columns):
Ticker                     505 non-null object
Price                      505 non-null float64
Number of Shares to Buy    505 non-null object
Price-to-Earnings Ratio    492 non-null object
PE Percentile              505 non-null object
Price-to-Book Ratio        487 non-null object
PB Percentile              505 non-null object
Price-to-Sales Ratio       487 non-null object
PS Percentile              505 non-null object
EV/EBITDA                  487 non-null float64
EV/EBITDA Percentile       505 non-null object
EV/GP                      487 non-null float64
EV/GP Percentile           505 non-null object
RV Score                   505 non-null object
dtypes: float64(3), object(11)
memory usage: 55.4+ KB


In [23]:
rv_dataframe.head()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,151.77,N/A,50.23,N/A,9.55,N/A,8.01,N/A,31.093367,N/A,14.956194,N/A,N/A
1,AAL,22.38,N/A,-1.51,N/A,-1.75,N/A,1.09,N/A,-3.819956,N/A,3.071992,N/A,N/A
2,AAP,220.63,N/A,22.87,N/A,3.98,N/A,1.32,N/A,12.187463,N/A,2.949457,N/A,N/A
3,AAPL,143.42,N/A,32.07,N/A,34.12,N/A,7.32,N/A,23.613407,N/A,18.221428,N/A,N/A
4,ABBV,120.84,N/A,40.87,N/A,15.14,N/A,4.14,N/A,13.038202,N/A,8.556115,N/A,N/A


### Dealing With Missing Data in Our DataFrame

In [24]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,240.16,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,279.83,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.8,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,30.08,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.26,N/A,14.45,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,63.86,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.61,N/A,33.59,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,36.81,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,191.52,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2680.39,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [25]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [26]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

In [27]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

    
rv_dataframe.head()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,151.77,N/A,50.23,0.813861,9.55,0.788119,8.01,0.813861,31.093367,0.829703,14.956194,0.813861,N/A
1,AAL,22.38,N/A,-1.51,0.118812,-1.75,0.0831683,1.09,0.10099,-3.819956,0.0376238,3.071992,0.134653,N/A
2,AAP,220.63,N/A,22.87,0.425743,3.98,0.522772,1.32,0.154455,12.187463,0.314851,2.949457,0.126733,N/A
3,AAPL,143.42,N/A,32.07,0.582178,34.12,0.958416,7.32,0.779208,23.613407,0.689109,18.221428,0.881188,N/A
4,ABBV,120.84,N/A,40.87,0.746535,15.14,0.873267,4.14,0.553465,13.038202,0.362376,8.556115,0.50099,N/A


## Calculating the RV Score

`Robust Value (RV)` will be the arithmetic mean of the 4 percentile scores.

In [28]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
        
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,151.77,N/A,50.23,0.813861,9.55,0.788119,8.01,0.813861,31.093367,0.829703,14.956194,0.813861,0.811881
1,AAL,22.38,N/A,-1.51,0.118812,-1.75,0.0831683,1.09,0.10099,-3.819956,0.0376238,3.071992,0.134653,0.0950495
2,AAP,220.63,N/A,22.87,0.425743,3.98,0.522772,1.32,0.154455,12.187463,0.314851,2.949457,0.126733,0.308911
3,AAPL,143.42,N/A,32.07,0.582178,34.12,0.958416,7.32,0.779208,23.613407,0.689109,18.221428,0.881188,0.77802
4,ABBV,120.84,N/A,40.87,0.746535,15.14,0.873267,4.14,0.553465,13.038202,0.362376,8.556115,0.50099,0.607327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.71,N/A,32.76,0.590099,-4.50,0.0811881,6.22,0.730693,23.588756,0.687129,10.255627,0.639604,0.545743
501,ZBH,169.64,N/A,60.70,0.867327,2.76,0.386139,4.96,0.619802,29.072681,0.8,8.334033,0.479208,0.630495
502,ZBRA,565.19,N/A,45.28,0.79604,12.13,0.831683,6.34,0.736634,33.363604,0.887129,13.790554,0.768317,0.80396
503,ZION,55.83,N/A,6.64,0.128713,1.20,0.131683,2.58,0.364356,5.201455,0.0554455,2.532811,0.10495,0.15703


## Removing Glamour Stocks

In [29]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)

rv_dataframe = rv_dataframe[:50]

rv_dataframe.reset_index(drop = True, inplace = True)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.98,N/A,7.81,0.132673,0.5701,0.0871287,0.4497,0.0217822,3.918428,0.0475248,0.457921,0.00990099,0.059802
1,MCK,199.32,N/A,-6.97,0.0990099,-1472.7100,0.0039604,0.1331,0.00990099,7.345367,0.0851485,2.506378,0.10099,0.059802
2,AIV,6.86,N/A,-76.52,0.0237624,2.0300,0.29901,-30.9000,0.0019802,-43.607345,0.00792079,-53.100114,0.00594059,0.0677228
3,AIG,50.42,N/A,-10.95,0.0871287,0.6710,0.0910891,0.9805,0.0871287,6.164896,0.0653465,0.973809,0.0237624,0.0708911
4,L,55.63,N/A,-400.40,0.0039604,0.8650,0.0990099,1.0500,0.0980198,8.143416,0.130693,1.026863,0.029703,0.0722772
5,HFC,33.66,N/A,-35.45,0.0475248,1.0400,0.110891,0.4790,0.0257426,8.111629,0.128713,2.737430,0.110891,0.0847525
6,AAL,22.38,N/A,-1.51,0.118812,-1.7500,0.0831683,1.0900,0.10099,-3.819956,0.0376238,3.071992,0.134653,0.0950495
7,ABC,120.87,N/A,-6.50,0.10297,-87.5900,0.00990099,0.1260,0.00792079,8.812900,0.152475,4.214985,0.215842,0.0978218
8,BA,239.50,N/A,-11.73,0.0831683,-8.0000,0.0376238,2.4600,0.350495,-19.151926,0.019802,-41.977783,0.00792079,0.099802
9,HPQ,31.10,N/A,11.28,0.180198,-11.1500,0.0316832,0.6090,0.039604,7.852574,0.108911,3.301869,0.150495,0.102178


## Number of Shares to Buy

In [30]:
# Portfolio size = 1 million US$.
portfolio_size = 1000000

In [31]:
# Top 50 stocks are equally weighted.
position_size = float(portfolio_size) / len(rv_dataframe.index)

for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])

rv_dataframe

c:\users\si\stocksselection\my_env\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.98,690,7.81,0.132673,0.5701,0.0871287,0.4497,0.0217822,3.918428,0.0475248,0.457921,0.00990099,0.059802
1,MCK,199.32,100,-6.97,0.0990099,-1472.7100,0.0039604,0.1331,0.00990099,7.345367,0.0851485,2.506378,0.10099,0.059802
2,AIV,6.86,2915,-76.52,0.0237624,2.0300,0.29901,-30.9000,0.0019802,-43.607345,0.00792079,-53.100114,0.00594059,0.0677228
3,AIG,50.42,396,-10.95,0.0871287,0.6710,0.0910891,0.9805,0.0871287,6.164896,0.0653465,0.973809,0.0237624,0.0708911
4,L,55.63,359,-400.40,0.0039604,0.8650,0.0990099,1.0500,0.0980198,8.143416,0.130693,1.026863,0.029703,0.0722772
5,HFC,33.66,594,-35.45,0.0475248,1.0400,0.110891,0.4790,0.0257426,8.111629,0.128713,2.737430,0.110891,0.0847525
6,AAL,22.38,893,-1.51,0.118812,-1.7500,0.0831683,1.0900,0.10099,-3.819956,0.0376238,3.071992,0.134653,0.0950495
7,ABC,120.87,165,-6.50,0.10297,-87.5900,0.00990099,0.1260,0.00792079,8.812900,0.152475,4.214985,0.215842,0.0978218
8,BA,239.50,83,-11.73,0.0831683,-8.0000,0.0376238,2.4600,0.350495,-19.151926,0.019802,-41.977783,0.00792079,0.099802
9,HPQ,31.10,643,11.28,0.180198,-11.1500,0.0316832,0.6090,0.039604,7.852574,0.108911,3.301869,0.150495,0.102178


## Export Recommended Trades

In [32]:
# Initializing XlsxWriter Object.
writer = pd.ExcelWriter('../recommended_trades/value_strategy_2.xlsx', engine='xlsxwriter')

rv_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [33]:
# Column formats.
# Background color.
background_color = '#0a0a23'

# Font color.
font_color = '#ffffff'

# String.
string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Price.
dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Integer.
integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Float.
float_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Percent.
percent_format = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [34]:
# Apply column formats.
column_formats = {
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Number of Shares to Buy', integer_format],
                    'D': ['Price-to-Earnings Ratio', float_format],
                    'E': ['PE Percentile', percent_format],
                    'F': ['Price-to-Book Ratio', float_format],
                    'G': ['PB Percentile',percent_format],
                    'H': ['Price-to-Sales Ratio', float_format],
                    'I': ['PS Percentile', percent_format],
                    'J': ['EV/EBITDA', float_format],
                    'K': ['EV/EBITDA Percentile', percent_format],
                    'L': ['EV/GP', float_format],
                    'M': ['EV/GP Percentile', percent_format],
                    'N': ['RV Score', percent_format]
                 }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [35]:
# Save the excel file.
writer.save()